# Compare ISO

Compare currency information coming from ISO site with the implementation of the library.
> Remember to compile the library before running the book


In [42]:
#r "../src/NMoneys/bin/Debug/net6.0/NMoneys.dll"

using NMoneys;
using NMoneys.Support;

using System;
using System.Globalization;
using System.Net.Http;
using System.Threading;
using System.Xml.Linq;

Microsoft.DotNet.Interactive.Formatting.Formatter.ListExpansionLimit= 200;

string url = "https://www.six-group.com/dam/download/financial-information/data-center/iso-currrency/lists/list-one.xml";
//string url = "http://127.0.0.1:5500/tools/response.xml";

record IsoCurrency(string Name, string AlphabeticCode, short NumericCode, byte? MinorUnits, bool Obsolete = false)
{
	public static IsoCurrency BuildFrom(Currency currency) =>
		new IsoCurrency(currency.EnglishName, currency.AlphabeticCode, currency.NumericCode, currency.SignificantDecimalDigits, currency.IsObsolete);

	public static IsoCurrency BuildFrom(XElement country)
	{
		short numericCode = short.Parse(country.Element("CcyNbr").Value);
		byte? minorUnits = byte.TryParse(country.Element("CcyMnrUnts").Value, out byte minor) ? minor : null;

		return new IsoCurrency(
			country.Element("CcyNm").Value,
			country.Element("Ccy").Value,
			numericCode,
			minorUnits
		);
	}
}

Dictionary<string, IsoCurrency> scrapped = new (StringComparer.Ordinal);

In [43]:
using (HttpClient client = new HttpClient())
{
	var source = new CancellationTokenSource();
	var stream = await client.GetStreamAsync(url, source.Token);
	XDocument doc = await XDocument.LoadAsync(stream, LoadOptions.None, source.Token);
	
	scrapped = doc.Root
		.Element("CcyTbl")
		.Elements("CcyNtry")
		.Where(e => e.Element("Ccy") is not null)
		.Select(IsoCurrency.BuildFrom)
		.Aggregate(scrapped, (acc, c) =>
		{
			if (acc.TryGetValue(c.AlphabeticCode, out IsoCurrency existing))
			{
				if (!existing.Equals(c)) Console.WriteLine($"In-file discrepancy: Existing {existing} | Incoming {c}");
			}
			else
			{
				acc.Add(c.AlphabeticCode, c);
			}
			return acc;
		});
}

In [44]:
var implemented = Currency.FindAll()
	.Select(IsoCurrency.BuildFrom)
	.ToDictionary(c => c.AlphabeticCode, c => c, StringComparer.Ordinal);

## Implemented-Only

Those currencies that are **NOT** coming from the ISO list but are implemented in the library.

They should be majorily obsolete.

In [45]:
implemented.Where(i => !scrapped.ContainsKey(i.Key))


index value 0 [HRK, IsoCurrency { Name = Kuna, AlphabeticCode = HRK, NumericCode = 191, MinorUnits = 2, Obsolete = False }] Key HRK Value IsoCurrency { Name = Kuna, AlphabeticCode = HRK, NumericCode = 191, MinorUnits = 2, Obsolete = False } Name Kuna AlphabeticCode HRK NumericCode 191 MinorUnits 2 Obsolete False 1 [EEK, IsoCurrency { Name = Estonian Kroon, AlphabeticCode = EEK, NumericCode = 233, MinorUnits = 2, Obsolete = True }] Key EEK Value IsoCurrency { Name = Estonian Kroon, AlphabeticCode = EEK, NumericCode = 233, MinorUnits = 2, Obsolete = True } Name Estonian Kroon AlphabeticCode EEK NumericCode 233 MinorUnits 2 Obsolete True 2 [LVL, IsoCurrency { Name = Latvian Lats, AlphabeticCode = LVL, NumericCode = 428, MinorUnits = 2, Obsolete = True }] Key LVL Value IsoCurrency { Name = Latvian Lats, AlphabeticCode = LVL, NumericCode = 428, MinorUnits = 2, Obsolete = True } Name Latvian Lats AlphabeticCode LVL NumericCode 428 MinorUnits 2 Obsolete True 3 [LTL, IsoCurrency { Name = Lithuanian Litas, AlphabeticCode = LTL, NumericCode = 440, MinorUnits = 2, Obsolete = True }] Key LTL Value IsoCurrency { Name = Lithuanian Litas, AlphabeticCode = LTL, NumericCode = 440, MinorUnits = 2, Obsolete = True } Name Lithuanian Litas AlphabeticCode LTL NumericCode 440 MinorUnits 2 Obsolete True 4 [MRO, IsoCurrency { Name = Ouguiya, AlphabeticCode = MRO, NumericCode = 478, MinorUnits = 2, Obsolete = True }] Key MRO Value IsoCurrency { Name = Ouguiya, AlphabeticCode = MRO, NumericCode = 478, MinorUnits = 2, Obsolete = True } Name Ouguiya AlphabeticCode MRO NumericCode 478 MinorUnits 2 Obsolete True 5 [STD, IsoCurrency { Name = Dobra, AlphabeticCode = STD, NumericCode = 678, MinorUnits = 2, Obsolete = True }] Key STD Value IsoCurrency { Name = Dobra, AlphabeticCode = STD, NumericCode = 678, MinorUnits = 2, Obsolete = True } Name Dobra AlphabeticCode STD NumericCode 678 MinorUnits 2 Obsolete True 6 [SLL, IsoCurrency { Name = Leone, AlphabeticCode = SLL, NumericCode = 694, MinorUnits = 2, Obsolete = False }] Key SLL Value IsoCurrency { Name = Leone, AlphabeticCode = SLL, NumericCode = 694, MinorUnits = 2, Obsolete = False } Name Leone AlphabeticCode SLL NumericCode 694 MinorUnits 2 Obsolete False 7 [ZMK, IsoCurrency { Name = Zambian Kwacha, AlphabeticCode = ZMK, NumericCode = 894, MinorUnits = 2, Obsolete = True }] Key ZMK Value IsoCurrency { Name = Zambian Kwacha, AlphabeticCode = ZMK, NumericCode = 894, MinorUnits = 2, Obsolete = True } Name Zambian Kwacha AlphabeticCode ZMK NumericCode 894 MinorUnits 2 Obsolete True 8 [VEF, IsoCurrency { Name = Bolívar, AlphabeticCode = VEF, NumericCode = 937, MinorUnits = 2, Obsolete = True }] Key VEF Value IsoCurrency { Name = Bolívar, AlphabeticCode = VEF, NumericCode = 937, MinorUnits = 2, Obsolete = True } Name Bolívar AlphabeticCode VEF NumericCode 937 MinorUnits 2 Obsolete True 9 [BYR, IsoCurrency { Name = Belarussian Ruble, AlphabeticCode = BYR, NumericCode = 974, MinorUnits = 0, Obsolete = True }] Key BYR Value IsoCurrency { Name = Belarussian Ruble, AlphabeticCode = BYR, NumericCode = 974, MinorUnits = 0, Obsolete = True } Name Belarussian Ruble AlphabeticCode BYR NumericCode 974 MinorUnits 0 Obsolete True 10 [USS, IsoCurrency { Name = US Dollar (Same day), AlphabeticCode = USS, NumericCode = 998, MinorUnits = 2, Obsolete = True }] Key USS Value IsoCurrency { Name = US Dollar (Same day), AlphabeticCode = USS, NumericCode = 998, MinorUnits = 2, Obsolete = True } Name US Dollar (Same day) AlphabeticCode USS NumericCode 998 MinorUnits 2 Obsolete True

## ISO-Only

These currencies are **ONLY** coming from the ISO list and are not implemented in the library.

In [46]:
scrapped.Where(s => !implemented.ContainsKey(s.Key))

index value 0 [SLE, IsoCurrency { Name = Leone, AlphabeticCode = SLE, NumericCode = 925, MinorUnits = 2, Obsolete = False }] Key SLE Value IsoCurrency { Name = Leone, AlphabeticCode = SLE, NumericCode = 925, MinorUnits = 2, Obsolete = False } Name Leone AlphabeticCode SLE NumericCode 925 MinorUnits 2 Obsolete False 1 [VED, IsoCurrency { Name = Bolívar Soberano, AlphabeticCode = VED, NumericCode = 926, MinorUnits = 2, Obsolete = False }] Key VED Value IsoCurrency { Name = Bolívar Soberano, AlphabeticCode = VED, NumericCode = 926, MinorUnits = 2, Obsolete = False } Name Bolívar Soberano AlphabeticCode VED NumericCode 926 MinorUnits 2 Obsolete False

## ISO Discrepancies

There currencies have different information in the ISO list and the library.

In [47]:
// TODO: use html tables to display information instead of JSON
record Diff(string Prop, object Scrapped, object Implemented);
Dictionary<string, Diff[]> discrepancies = new(StringComparer.Ordinal);
StringComparer nameComparer = StringComparer.Create(CultureInfo.GetCultureInfo("en-GB"), false);
foreach (IsoCurrency iso in scrapped.Values)
{
	if (implemented.TryGetValue(iso.AlphabeticCode, out IsoCurrency alsoImplemented))
	{
		List<Diff> diffs = new(3);
		if (!iso.NumericCode.Equals(alsoImplemented.NumericCode))
		{
			diffs.Add(new Diff("NumericCode", iso.NumericCode, alsoImplemented.NumericCode));
		}
		if (!nameComparer.Equals(iso.Name, alsoImplemented.Name))
		{
			diffs.Add(new Diff("Name", iso.Name, alsoImplemented.Name));
		}
		if (!iso.MinorUnits.Equals(alsoImplemented.MinorUnits))
		{
			diffs.Add(new Diff("MinorUnits", iso.MinorUnits, alsoImplemented.MinorUnits));
		}
		if (diffs.Count > 0)
		{
			discrepancies.Add(iso.AlphabeticCode, diffs.ToArray());
		}
	}
}
discrepancies.Display();

key value KMF index value 0 Diff { Prop = Name, Scrapped = Comorian Franc , Implemented = Comorian Franc } Prop Name Scrapped Comorian Franc Implemented Comorian Franc XDR index value 0 Diff { Prop = MinorUnits, Scrapped = , Implemented = 2 } Prop MinorUnits Scrapped <null> Implemented 2 XUA index value 0 Diff { Prop = MinorUnits, Scrapped = , Implemented = 2 } Prop MinorUnits Scrapped <null> Implemented 2 XSU index value 0 Diff { Prop = MinorUnits, Scrapped = , Implemented = 2 } Prop MinorUnits Scrapped <null> Implemented 2 UYI index value 0 Diff { Prop = Name, Scrapped = Uruguay Peso en Unidades Indexadas (UI), Implemented = Uruguay Peso en Unidades Indexadas (URUIURUI) } Prop Name Scrapped Uruguay Peso en Unidades Indexadas (UI) Implemented Uruguay Peso en Unidades Indexadas (URUIURUI) UYW index value 0 Diff { Prop = Name, Scrapped = Unidad Previsional, Implemented = Unidad previsional } Prop Name Scrapped Unidad Previsional Implemented Unidad previsional 1 Diff { Prop = MinorUnits, Scrapped = 4, Implemented = 0 } Prop MinorUnits Scrapped 4 Implemented 0 XBA index value 0 Diff { Prop = Name, Scrapped = Bond Markets Unit European Composite Unit (EURCO), Implemented = European Composite Unit (EURCO) } Prop Name Scrapped Bond Markets Unit European Composite Unit (EURCO) Implemented European Composite Unit (EURCO) 1 Diff { Prop = MinorUnits, Scrapped = , Implemented = 2 } Prop MinorUnits Scrapped <null> Implemented 2 XBB index value 0 Diff { Prop = Name, Scrapped = Bond Markets Unit European Monetary Unit (E.M.U.-6), Implemented = European Monetary Unit (E.M.U.-6) } Prop Name Scrapped Bond Markets Unit European Monetary Unit (E.M.U.-6) Implemented European Monetary Unit (E.M.U.-6) 1 Diff { Prop = MinorUnits, Scrapped = , Implemented = 2 } Prop MinorUnits Scrapped <null> Implemented 2 XBC index value 0 Diff { Prop = Name, Scrapped = Bond Markets Unit European Unit of Account 9 (E.U.A.-9), Implemented = European Unit of Account 9 (E.U.A.-9) } Prop Name Scrapped Bond Markets Unit European Unit of Account 9 (E.U.A.-9) Implemented European Unit of Account 9 (E.U.A.-9) 1 Diff { Prop = MinorUnits, Scrapped = , Implemented = 2 } Prop MinorUnits Scrapped <null> Implemented 2 XBD index value 0 Diff { Prop = Name, Scrapped = Bond Markets Unit European Unit of Account 17 (E.U.A.-17), Implemented = European Unit of Account 17 (E.U.A.-17) } Prop Name Scrapped Bond Markets Unit European Unit of Account 17 (E.U.A.-17) Implemented European Unit of Account 17 (E.U.A.-17) 1 Diff { Prop = MinorUnits, Scrapped = , Implemented = 2 } Prop MinorUnits Scrapped <null> Implemented 2 XTS index value 0 Diff { Prop = Name, Scrapped = Codes specifically reserved for testing purposes, Implemented = Test currency } Prop Name Scrapped Codes specifically reserved for testing purposes Implemented Test currency 1 Diff { Prop = MinorUnits, Scrapped = , Implemented = 2 } Prop MinorUnits Scrapped <null> Implemented 2 XXX index value 0 Diff { Prop = Name, Scrapped = The codes assigned for transactions where no currency is involved, Implemented = No currency } Prop Name Scrapped The codes assigned for transactions where no currency is involved Implemented No currency 1 Diff { Prop = MinorUnits, Scrapped = , Implemented = 2 } Prop MinorUnits Scrapped <null> Implemented 2 XAU index value 0 Diff { Prop = MinorUnits, Scrapped = , Implemented = 2 } Prop MinorUnits Scrapped <null> Implemented 2 XPD index value 0 Diff { Prop = MinorUnits, Scrapped = , Implemented = 2 } Prop MinorUnits Scrapped <null> Implemented 2 XPT index value 0 Diff { Prop = MinorUnits, Scrapped = , Implemented = 2 } Prop MinorUnits Scrapped <null> Implemented 2 XAG index value 0 Diff { Prop = MinorUnits, Scrapped = , Implemented = 2 } Prop MinorUnits Scrapped <null> Implemented 2